### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt 
# import matplotlib.cm as cm
# from adjustText import adjust_text 
# import itertools
# import re

###  Datasets to be studied
 I) Confirmed Cases Dataset of COVID-19
 
 II) Death Cases Dataset of COVID-19
 
 III) Recovered Cases Dataset of COVID-19
 
 IV)Testing Dataset of COVID-19
 
 V) INFORM Risk Index dataset

#### I) Confirmed Cases of COVID-19 

In [2]:
# Importing  Confirmed Cases Dataset
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_confirmed = pd.read_csv(url_confirmed, index_col="Country/Region")
df_confirmed.drop(['Lat', 'Long'], axis=1, inplace=True)

##### Data Preprocessing for the dataset I

# Country grouping on original dataframe        
gr_confirmed = df_confirmed.groupby("Country/Region").sum()
gr_confirmed.head(3)

# Adding data for China (1-Jan to 20-Jan from a China CDC publication)
lab=[]
for i in range(1,22):
    lab.append("1/" + str(i) + "/20")
    gr_confirmed.insert(loc=i-1,column=lab[i-1], value=0)
gr_confirmed.loc["China"][0:10] = 20
gr_confirmed.loc["China"][10:21] = 310

# Remove Diamond princess
gr_confirmed = gr_confirmed.drop("Diamond Princess")

# Adding the prefix to  all columns of Confirmed Cases columns, which are dates to distingish them from
# dates under Deaths dataset columns which are also same dates.
# gr_confirmed = gr_confirmed.add_prefix('Confirmed Cases on ')
# gr_confirmed.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Confirmed.csv")

gr_confirmed1= gr_confirmed.copy() # To have a copy of Confirmed Cases dataset in date format because ahead we are going to transform the data to days format
gr_confirmed1 = gr_confirmed1.reset_index()
gr_confirmed1.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Confirmed_datewise.csv")

gr_confirmed.head(3)

,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,1/10/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653
Albania,0,0,0,0,0,0,0,0,0,0,...,868,872,876,880,898,916,933,946,948,949
Algeria,0,0,0,0,0,0,0,0,0,0,...,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377


In [3]:
# Checking for the missing values in the Confirmed Cases dataset

gr_confirmed.isnull().sum().sum()
gr_confirmed.isna().sum().sum()

# Hence, there are no missing values in our Confirmed Cases data. 

0

#### Shifting all data to one starting point

#### Converting the Dates format to Days to bring all countries data to start from 1 starting point "Day1: 1"
Day1: 1,  is the day on which the confirmed cases for the country got greater than or equal to 5 

In [4]:
# Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
gr_confirmed_melt = gr_confirmed
dates = gr_confirmed_melt.keys()
FMT = '%m/%d/%y'

days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("1/1/20", FMT)).days)

days = days + 1
# +1 is to start the days from 1 instead of 0

gr_confirmed_melt.columns = days # = dates will convert the columns to date formats again

x_lag = gr_confirmed_melt.ge(5).idxmax(axis=1) # x_lag gives position of first value in each row which is greater than or equal to 5

for i in range(gr_confirmed_melt.shape[0]): # gr_confirmed_melt.shape[0] = 187 (no. of rows) and gr_confirmed_melt.shape[1] = 138 (no. of columns)
    gr_confirmed_melt.iloc[i] = gr_confirmed_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
gr_confirmed_melt.head(8)

# Melting our Confirmed Cases dataset

gr_confirmed_melt = gr_confirmed_melt.reset_index()
gr_confirmed_melt = pd.melt(gr_confirmed_melt,id_vars= "Country/Region", 
                       value_vars=days, var_name="Days", 
                       value_name="Confirmed Cases").sort_values(["Country/Region","Confirmed Cases"])

gr_confirmed_melt.set_index("Country/Region", inplace = True)
gr_confirmed_melt.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Confirmed_daywise_melted.csv")
# gr_confirmed_melt.shape
gr_confirmed_melt.head(3)

,Days,Confirmed Cases
Country/Region,,
Afghanistan,1,5.0
Afghanistan,2,7.0
Afghanistan,3,7.0


#### II) Deaths due to COVID-19 

In [5]:
# Importing Deaths Dataset
url_death = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_death = pd.read_csv(url_death, index_col="Country/Region")
df_death.drop(['Lat', 'Long'], axis=1, inplace=True)
df_death.head(3)

#df_death.to_csv("C:/Users/user/Downloads/COVID-19 Related info/time_series_COVID19-death.csv")

##### Data Preprocessing for the dataset II

# Country grouping on original dataframe        
gr_death = df_death.groupby("Country/Region").sum()
gr_death.head(3)

# # Now adding Australian dataframe
# gr_death = pd.concat([gr_death, Aust_death])

# Adding data for China (1-Jan to 20-Jan) 
lab=[]
for i in range(1,22):
    lab.append("1/" + str(i) + "/20")
    gr_death.insert(i-1,lab[i-1],0)
gr_death.loc["China"][0:10] = 1
gr_death.loc["China"][10:21] = 1

# Remove Diamond princess
gr_death = gr_death.drop("Diamond Princess")

# Adding the prefix to  all columns of Deaths dataset columns, which are dates to distingish them 
# from dates under Confirmed Cases dataset columns which are also same dates.
# gr_death = gr_death.add_prefix("Deaths on ")
# gr_death.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Deaths.csv")

gr_death1 = gr_death.copy() # To have a copy of death dataset in date format because ahead we are going to transform the data to days format
gr_death1 = gr_death1.reset_index()
gr_death1.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Deaths_datewise.csv")

gr_death.head(3)

,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,1/10/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,120,122,127,132,136,153,168,169,173,178
Albania,0,0,0,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
Algeria,0,0,0,0,0,0,0,0,0,0,...,502,507,515,522,529,536,542,548,555,561


In [6]:
# Checking for the missing values in the death Cases dataset

gr_death.isnull().sum().sum()
gr_death.isna().sum().sum()

# Hence, there are no missing values in our death Cases data. 

0

In [7]:
##### Shifting all data to one starting point

# # Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
# gr_death_melt = gr_death
# dates = gr_death_melt.keys()
# FMT = '%m/%d/%y'

# days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("1/1/20", FMT)).days)

# days = days + 1
# # +1 is to start the days from 1 instead of 0

# gr_death_melt.columns = days # = dates will convert the columns to date formats again

# x_lag = gr_death_melt.ge(5).idxmax(axis=1) # x_lag gives position of first value in each row which is greater than or equal to 5

# for i in range(gr_death_melt.shape[0]): # gr_death_melt.shape[0] = 187 (no. of rows) and gr_death_melt.shape[1] = 138 (no. of columns)
#     gr_death_melt.iloc[i] = gr_death_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
# # gr_death_melt.head(3)

# # Melting our Confirmed Cases dataset

# gr_death_melt = gr_death_melt.reset_index()
# gr_death_melt = pd.melt(gr_death_melt,id_vars= "Country/Region", 
#                        value_vars=days, var_name="Days", 
#                        value_name="Deaths").sort_values(["Country/Region","Deaths"])

# gr_death_melt.set_index("Country/Region", inplace = True)
# gr_death_melt.head(3)

#### Shifting all data to one starting point
#### Converting the Dates format to Days to bring all countries data to start from 1 starting point "Day1: 1"
Day1: 1,  is the day on which the confirmed cases for the country got greater than or equal to 5 

In [8]:
# Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
gr_death_melt = gr_death
dates = gr_death_melt.keys()
FMT = '%m/%d/%y'

days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("1/1/20", FMT)).days)
days = days + 1
# +1 is to start the days from 1 instead of 0

gr_death_melt.columns = days # = dates will convert the columns to date formats again

x_lag  # x_lag gives position of first value in each row which is greater than or equal to 5 in confirmed cases dataset

for i in range(gr_death_melt.shape[0]): # gr_death_melt.shape[0] = 187 (no. of rows) and gr_death_melt.shape[1] = 138 (no. of columns)
    gr_death_melt.iloc[i] = gr_death_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
gr_death_melt.head(8)

# Melting our Confirmed Cases dataset

gr_death_melt = gr_death_melt.reset_index()
gr_death_melt = pd.melt(gr_death_melt,id_vars= "Country/Region", 
                       value_vars=days, var_name="Days", 
                       value_name="Deaths").sort_values(["Country/Region","Deaths"])

gr_death_melt.set_index("Country/Region", inplace = True)
gr_death_melt.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Deaths_daywise_melted.csv")
# gr_death_melt.shape
gr_death_melt.head(3)

,Days,Deaths
Country/Region,,
Afghanistan,1,0.0
Afghanistan,2,0.0
Afghanistan,3,0.0


#### III) Recovered Cases of COVID-19

In [9]:
# Importing dataset for Recovered Cases
url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
df_recovered = pd.read_csv(url_recovered, index_col="Country/Region")
df_recovered.drop(['Lat', 'Long'], axis=1, inplace=True)
df_recovered.head(3)

##### Data Prepocessing for dataset IV

# Country grouping on original dataframe        
gr_recovered = df_recovered.groupby("Country/Region").sum()

# Now adding Australian dataframe
# gr_death = pd.concat([gr_death, Aust_death])

# Adding data for China (1-Jan to 20-Jan) 
lab=[]
for i in range(1,22):
    lab.append("1/" + str(i) + "/20")
    gr_recovered.insert(i-1,lab[i-1],0)
gr_recovered.loc["China"][0:10] = 1
gr_recovered.loc["China"][10:21] = 1

# Remove Diamond princess
gr_recovered = gr_recovered.drop("Diamond Princess")

# Adding the prefix to  all columns of Deaths dataset columns, which are dates to distingish them 
# from dates under Confirmed Cases dataset columns which are also same dates.
# gr_recovered = gr_recovered.add_prefix("Recovered Cases on ")
# gr_recovered.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Recovered.csv")
gr_recovered1 = gr_recovered.copy()  # To have a copy of Recovered Cases dataset in date format because ahead we are going to transform the data to days format
gr_recovered1 = gr_recovered1.reset_index()
gr_recovered1.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Recovered_datewise.csv")

# gr_recovered.shape
gr_recovered.head(3)

,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,1/10/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,558,558,610,648,691,745,745,778,801,850
Albania,0,0,0,0,0,0,0,0,0,0,...,650,654,682,688,694,705,714,715,727,742
Algeria,0,0,0,0,0,0,0,0,0,0,...,2678,2841,2998,3058,3158,3271,3409,3507,3625,3746


In [10]:
# Checking for the missing values in the Recovered Cases dataset

gr_recovered.isnull().sum().sum()
gr_recovered.isna().sum().sum()

# Hence, there are no missing values in our Recovered Cases data. 

0

In [11]:
##### Shifting all data to one starting point

# # Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
# gr_recovered_melt = gr_recovered
# dates = gr_recovered_melt.keys()
# FMT = '%m/%d/%y'

# days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("1/1/20", FMT)).days)

# days = days + 1
# # +1 is to start the days from 1 instead of 0

# gr_recovered_melt.columns = days # = dates will convert the columns to date formats again

# x_lag = gr_recovered_melt.ge(5).idxmax(axis=1) # x_lag gives position of first value in each row which is greater than or equal to 5

# for i in range(gr_recovered_melt.shape[0]): # gr_recovered_melt.shape[0] = 187 (no. of rows) and gr_recovered_melt.shape[1] = 138 (no. of columns)
#     gr_recovered_melt.iloc[i] = gr_recovered_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
# gr_recovered_melt.head(3)

# #### Melting our Confirmed Cases dataset

# gr_recovered_melt = gr_recovered_melt.reset_index()
# gr_recovered_melt = pd.melt(gr_recovered_melt,id_vars= "Country/Region", 
#                        value_vars=days, var_name="Days", 
#                        value_name="Recovered Cases").sort_values(["Country/Region","Recovered Cases"])

# gr_recovered_melt.set_index("Country/Region", inplace = True)
# gr_recovered_melt.head(3)

#### Shifting all data to one starting point
#### Converting the Dates format to Days to bring all countries data to start from 1 starting point "Day1: 1"
Day1: 1,  is the day on which the confirmed cases for the country got greater than or equal to 5 

In [12]:
# Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
gr_recovered_melt = gr_recovered
dates = gr_recovered_melt.keys()
FMT = '%m/%d/%y'

days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("1/1/20", FMT)).days)

days = days + 1
# +1 is to start the days from 1 instead of 0

gr_recovered_melt.columns = days # = dates will convert the columns to date formats again

x_lag # x_lag gives position of first value in each row which is greater than or equal to 5 in confirmed cases data

for i in range(gr_recovered_melt.shape[0]): # gr_recovered_melt.shape[0] = 187 (no. of rows) and gr_recovered_melt.shape[1] = 138 (no. of columns)
    gr_recovered_melt.iloc[i] = gr_recovered_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
gr_recovered_melt.head(3)

#### Melting our Confirmed Cases dataset

gr_recovered_melt = gr_recovered_melt.reset_index()
gr_recovered_melt = pd.melt(gr_recovered_melt,id_vars= "Country/Region", 
                       value_vars=days, var_name="Days", 
                       value_name="Recovered Cases").sort_values(["Country/Region","Recovered Cases"])

gr_recovered_melt.set_index("Country/Region", inplace = True)
gr_recovered_melt.to_csv("C:/Users/user/Downloads/COVID-19 Related Info/Created Files/Recovered_daywise_melted.csv")
# gr_recovered_melt.shape
gr_recovered_melt.head(3)

,Days,Recovered Cases
Country/Region,,
Afghanistan,1,0.0
Afghanistan,2,0.0
Afghanistan,3,0.0


#### IV) Importing Dataset for Testing of COVID-19 

In [13]:
url_test= 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv'
        
df_test = pd.read_csv(url_test)
#                       , index_col="Entity")
df_test.head(2)

,Entity,ISO code,Date,Source URL,Source label,Notes,Cumulative total,Daily change in cumulative total,Cumulative total per thousand,Daily change in cumulative total per thousand,7-day smoothed daily change,7-day smoothed daily change per thousand
0,Argentina - tests performed,ARG,2020-04-08,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,13330.0,NaN,0.295,NaN,NaN,NaN
1,Argentina - tests performed,ARG,2020-04-09,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,14850.0,1520.0,0.329,0.034,NaN,NaN


##### Data Preprocessing for the dataset IV

In [14]:
df_test.shape

(6192, 12)

In [15]:
df_test.columns

Index(['Entity', 'ISO code', 'Date', 'Source URL', 'Source label', 'Notes',
       'Cumulative total', 'Daily change in cumulative total',
       'Cumulative total per thousand',
       'Daily change in cumulative total per thousand',
       '7-day smoothed daily change',
       '7-day smoothed daily change per thousand'],
      dtype='object')

###### 1. Removing the not required columns for the analysis from the testing data

In [16]:
df_test.drop(['Source URL', 'Source label', 'Notes','Daily change in cumulative total',
       'Cumulative total per thousand',
       'Daily change in cumulative total per thousand',
       '7-day smoothed daily change',
       '7-day smoothed daily change per thousand', 'ISO code'], axis=1, inplace=True)

In [17]:
df_test.head(3)

,Entity,Date,Cumulative total
0,Argentina - tests performed,2020-04-08,13330.0
1,Argentina - tests performed,2020-04-09,14850.0
2,Argentina - tests performed,2020-04-10,16379.0


##### 2) Changing the "Entity" column to make it more useful for our analysis

In [18]:
# When we groupby on a column it becomes the index for the dataframe. Here we did group_by on "Entity", so it is the index for
# the resuting dataframe now.

# Since here the name of the country column is "Entity", which is different from the name of te country column "Country/Region"
# in confirmed cases and deaths dataset
# Also Since the country names are concatenated with strings "- tests performed" which we need to separate from name of the country 
# names in that string. 
# So to perform both of these functions we choose to change the index column  "Entity" to a normal column and then peform the 
# required changes on it and then after applying the changes we can reset it to index again.

In [19]:
# Do the following:
# a) Change the name of the index column to "Country/Region" as in Confirmed Cases/Deaths and Recovered Cases datasets
# b) Get the country name from the value under Entity column like: "Argentina" from "Argentina - tests performed"

In [20]:
# a) Changing the name of the column

# Modifying the name of the "Entity" column to "Country/Region" as in confirmed and deaths datasets to join them using same 
# column with same name

df_test= df_test.rename(columns={"Entity":"Country/Region", "Cumulative total": "Cumulative_Testing_Count"})
# df_test.columns
df_test.head(3)

,Country/Region,Date,Cumulative_Testing_Count
0,Argentina - tests performed,2020-04-08,13330.0
1,Argentina - tests performed,2020-04-09,14850.0
2,Argentina - tests performed,2020-04-10,16379.0


In [21]:
# b) To get the country name from the value under Entity column like: "Argentina" from "Argentina - tests performed"
df_test["Country/Region"]= df_test["Country/Region"].str.split(" -", n=1, expand=True)
df_test.head(3)

,Country/Region,Date,Cumulative_Testing_Count
0,Argentina,2020-04-08,13330.0
1,Argentina,2020-04-09,14850.0
2,Argentina,2020-04-10,16379.0


##### 3) Changing the "Date" column from object type to type datetime and then changing it to the format present in Confirmed Cases/ Deaths/Recovered Cases datasets for "Dates"

In [22]:
# type(df_test["Date"])
df_test.dtypes

Country/Region               object
Date                         object
Cumulative_Testing_Count    float64
dtype: object

In [23]:
df_test['Date'] = df_test['Date'].astype('datetime64[ns]')    

df_test.dtypes

Country/Region                      object
Date                        datetime64[ns]
Cumulative_Testing_Count           float64
dtype: object

In [24]:
# Changing the format of the "Date" column to the one matching the dates in Confirmed Cases/Deaths and Recovered Cases datasets.
df_test['Date'] = df_test['Date'].dt.strftime('%m/%d/%y')
df_test.dtypes

Country/Region               object
Date                         object
Cumulative_Testing_Count    float64
dtype: object

##### 4) Unmelting the Testing data such that the individual dates form the separate columns

In [25]:
df_test_unmelted = df_test.pivot_table(index="Country/Region", columns='Date')
df_test_unmelted = df_test_unmelted["Cumulative_Testing_Count"].reset_index()
df_test_unmelted.columns.name = None
df_test_unmelted.head(3)

,Country/Region,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,...,05/10/20,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20
0,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83018.0,85158.0,87547.0,90474.0,93673.0,96893.0,100362.0,103220.0,105829.0,NaN
1,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,827872.0,855119.0,877927.0,909025.0,943480.0,983816.0,1015652.0,1042126.0,1062034.0,1085870.0
2,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,316508.0,319484.0,329314.0,336252.0,344606.0,351351.0,357393.0,362509.0,365873.0,372435.0


##### 5) Filling Missing values with appropriate values

In [26]:
##### 5a)

# The testing data is not available for all dates for the countries, even after getting first non empty observation for the 
# country for its testing data there are further dates for which no data is available.
# For example for Argenina testing data is available from Apr 8,2020 till present date but is in between missing for some dates
# like April 12, 2020, for Australia first observation on March 22, 2020 but no data till March 28th after that and similarly for 
# other countries. 
# So we will fill those empty values with the previous non-zero values as follows.

# df_test_unmelted.mask(df_test_unmelted.iloc[:, 1:] == 0).ffill(downcast='infer', axis=1, inplace = True)
df_test_unmelted.iloc[:, 1:] = df_test_unmelted.iloc[:, 1:].fillna(method='ffill', downcast='infer', axis=1)
df_test_unmelted.head(3)

,Country/Region,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,...,05/10/20,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20
0,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83018.0,85158.0,87547.0,90474.0,93673.0,96893.0,100362.0,103220.0,105829.0,105829.0
1,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,827872.0,855119.0,877927.0,909025.0,943480.0,983816.0,1015652.0,1042126.0,1062034.0,1085870.0
2,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,316508.0,319484.0,329314.0,336252.0,344606.0,351351.0,357393.0,362509.0,365873.0,372435.0


In [27]:
##### 5b)

# Now filling the remaining missing values which are before the first non-zero values for any country with zeroes

df_test_unmelted = df_test_unmelted.fillna(0)
df_test_unmelted.head(3)

,Country/Region,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,...,05/10/20,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20
0,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,83018.0,85158.0,87547.0,90474.0,93673.0,96893.0,100362.0,103220.0,105829.0,105829.0
1,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,827872.0,855119.0,877927.0,909025.0,943480.0,983816.0,1015652.0,1042126.0,1062034.0,1085870.0
2,Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,316508.0,319484.0,329314.0,336252.0,344606.0,351351.0,357393.0,362509.0,365873.0,372435.0


##### 6) Preparing the testing data to be merged with the "Confirmed Cases"/"Deaths"/"Recovered" Cases datasets by aligning the "Country/Region" column with other datasets 

In [28]:
# Comparing the Confirmed Cases (or/Deaths/ Recovered Cases) dataset with Testing data for different country names for the same
# country or different countries in the two datasets using the gr_confirmed1 file - the datewise dataset copy of confirmed cases
# and the unmelted testing dataset (df_test_unmelted)

countries_in_either_datasets = pd.merge(gr_confirmed1["Country/Region"], df_test_unmelted,  how='outer',on= "Country/Region", indicator = True)
# countries_in_either_datasets

countries_only_in_df_ConfirmedData =countries_in_either_datasets[countries_in_either_datasets['_merge'] == 'left_only']
countries_only_in_df_TestData = countries_in_either_datasets[countries_in_either_datasets['_merge'] == 'right_only']

countries_only_in_df_TestData
# countries_only_in_df_ConfirmedData

,Country/Region,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,...,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20,_merge
187,Czech Republic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,318979.0,327688.0,335375.0,342262.0,348965.0,353223.0,356614.0,363963.0,363963.0,right_only
188,Hong Kong,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,160055.0,183491.0,183491.0,183491.0,183491.0,183491.0,183491.0,183491.0,183491.0,right_only
189,Myanmar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11822.0,12062.0,12482.0,12995.0,13634.0,13999.0,14561.0,14561.0,15137.0,right_only
190,South Korea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,668492.0,680890.0,695920.0,711484.0,726747.0,740645.0,747653.0,753211.0,765574.0,right_only
191,Taiwan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,67758.0,68107.0,68335.0,68659.0,68988.0,69206.0,69395.0,69657.0,69657.0,right_only
192,United States,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9365286.0,9697459.0,10106980.0,10378046.0,10739591.0,11092529.0,11172133.0,11558644.5,11558644.5,right_only


In [29]:
# Doing this check we get to know that in gr_death and gr_confirmed and gr_recovered has 186 rows each and all values are common
# but in test_df there are just 82 rows out of which following rows are just in test_df and some values out of these are different 
# names for the same country presnt in the other 3 dataframes. So we rename those values and keep other which are not present
# in other 3 dataframes just like that, with NAs for their columns for those countries.

# gr_confirmed1 = gr_confirmed1.rename(index={'Taiwan*': 'Taiwan'})
# df_test_unmelted = df_test_unmelted.rename(index={'United States': 'US', "Czech Republic":"Czechia", "South Korea": "Korea, South"})

gr_confirmed1["Country/Region"].replace({'Taiwan*': 'Taiwan'}, inplace = True)
df_test_unmelted["Country/Region"].replace({'United States': 'US', "Czech Republic":"Czechia", "South Korea": "Korea, South"}, inplace = True)


In [30]:
# Making the Changes in original datasets of Confirmed Cases/ Deaths/ Recovered  datasets and their melted forms

gr_confirmed = gr_confirmed.rename(index={'Taiwan*': 'Taiwan'})
gr_death = gr_death.rename(index={'Taiwan*': 'Taiwan'})
gr_recovered = gr_recovered.rename(index={'Taiwan*': 'Taiwan'})

gr_confirmed_melt = gr_confirmed_melt.rename(index={'Taiwan*': 'Taiwan'})
gr_death_melt = gr_death_melt.rename(index={'Taiwan*': 'Taiwan'})
gr_recovered_melt = gr_recovered_melt.rename(index={'Taiwan*': 'Taiwan'})

In [31]:
# Since there are just 3 countries namely "Czech Republic","Hong Kong" and "Myanmar" which are "only" in Testing data and all 
# the other c"Country/Region" value in testing data are there in Confirmed/Deaths/Recovered cases datasets.

# So we leave these three countries for the purpose of analysis and keep only the "Country/Region" values which are in 
# both datasets (Confirmed/Deaths/Recovered cases dataset and Testing dataset) 

In [32]:
# To include all the countries which are there in Confirmed Cases/ Deaths/ Recovered Cases Datasets 
# By joining the "Country/Region"  column in Confirmed Cases dataset with the whole testing (unmelted) data by joining on
# common column "Country/Region"

# df_test_unmelted = pd.merge(gr_confirmed1['Country/Region'], df_test_unmelted, on='Country/Region', how='outer', indicator=True)
# df_test_unmelted = df_test_unmelted_1[df_test_unmelted_1['_merge'] == 'left_only']
df_test_unmelted = pd.merge(gr_confirmed1['Country/Region'], df_test_unmelted, on='Country/Region', how='left')

df_test_unmelted.head(3)

,Country/Region,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,...,05/10/20,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Filling the missing values in the rows("Contry/Region" values) which were earlier only in Confirmed/Deaths/Recovered Datasets
# but now after merging the "Country/Region" column in Confirmed Cases dataset, are there in testing data also with 0.

In [34]:
df_test_unmelted.fillna(0, inplace=True)
df_test_unmelted.set_index("Country/Region", inplace=True)
df_test_unmelted.to_csv("C:/Users/user/Downloads/COVID-19 Related info/Created Files/Testing_datewise.csv")
df_test_unmelted.head(3)

,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,01/10/20,...,05/10/20,05/11/20,05/12/20,05/13/20,05/14/20,05/15/20,05/16/20,05/17/20,05/18/20,05/19/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_test_unmelted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, Afghanistan to Zimbabwe
Columns: 140 entries, 01/01/20 to 05/19/20
dtypes: float64(140)
memory usage: 206.0+ KB


#### Shifting all data to one starting point
#### Converting the Dates format to Days to bring all countries data to start from 1 starting point "Day1: 1"
Day1: 1,  is the day on which the confirmed cases for the country got greater than or equal to 5 

In [36]:
# Dates are converted into no of days since 1/1/20 so that 1/1/20 corresponds to day 1
df_test_melt = df_test_unmelted
dates = df_test_melt.keys()
FMT = '%m/%d/%y'

days = dates.map(lambda x: (datetime.strptime(x, FMT) - datetime.strptime("01/01/20", FMT)).days)
days = days + 1
# +1 is to start the days from 1 instead of 0

df_test_melt.columns = days # = dates will convert the columns to date formats again

x_lag  # x_lag gives position of first value in each row which is greater than or equal to 5 in confirmed cases data

for i in range(df_test_melt.shape[0]): # gr_recovered_melt.shape[0] = 187 (no. of rows) and gr_recovered_melt.shape[1] = 138 (no. of columns)
    df_test_melt.iloc[i] = df_test_melt.iloc[i].shift(periods=-x_lag[i]+1) # all data shift to one starting point 
df_test_melt.head(3)

#### Melting our Confirmed Cases dataset

df_test_melt = df_test_melt.reset_index()
df_test_melt = pd.melt(df_test_melt,id_vars= "Country/Region", 
                       value_vars=days, var_name="Days", 
                       value_name="Testing Count").sort_values(["Country/Region","Testing Count"])

# df_test_melt.columns = dates
df_test_melt.set_index("Country/Region", inplace = True)
df_test_melt.to_csv("C:/Users/user/Downloads/COVID-19 Related info/Created Files/Testing_daywise_melted.csv")
# df_test_melt.shape
df_test_melt.head(3)

,Days,Testing Count
Country/Region,,
Afghanistan,1,0.0
Afghanistan,2,0.0
Afghanistan,3,0.0


##### V) Importing the INFORM- Risk Index data

In [37]:
df = pd.read_excel('C:/Users/user/Downloads/INFORM COVID-19 RISK INDEX v012.xlsx',sheet_name=None, skiprows= range(1))
# sheet_name=None implies all sheets
# Skipping first row because it is blank in all our sheets.)

#### Data Preprocessing for the data
##### which is in the form of collections.OrderedDict because of multiple worksheets imported

In [38]:
type(df)

collections.OrderedDict

In [39]:
len(df)

14

In [40]:
df.keys() # To get the names of the sheets in our Excel file and now in our OrderedDict df

odict_keys(['Home', 'Table of Contents', 'INFORM COVID-19 RISK 2020 (a-z)', 'Hazard & Exposure', 'Vulnerability', 'Lack of Coping Capacity', 'Indicator Data', 'Indicator Date hidden', 'Indicator Date hidden2', 'Indicator Source', 'Indicator Data imputation', 'Imputed and missing data hidden', 'Indicator Metadata', 'Regions'])

In [41]:
# We need only 4 sheets from all the 14 sheets. 'INFORM COVID-19 RISK 2020 (a-z)','Hazard & Exposure', 'Vulnerability', 
#'Lack of Coping Capacity'.So dropping all other not to be used sheets from df.

for e in ["Home",'Table of Contents', 'Indicator Data','Indicator Date hidden','Indicator Date hidden2', 'Indicator Source', 'Indicator Data imputation', 'Imputed and missing data hidden', 'Indicator Metadata', 'Regions']: 
    df.pop(e)

In [42]:
df.keys() # To get the names of the sheets in our OrderedDict df after dropping not to be used sheets

odict_keys(['INFORM COVID-19 RISK 2020 (a-z)', 'Hazard & Exposure', 'Vulnerability', 'Lack of Coping Capacity'])

In [43]:
# df.items() # to get list of all worksheets in our df orderedDict

# print(list(df).index('Home')) doesn't return 0 (its index in df)
# df.items()[i] i=0,1...13; returns error
# because In Python 3, dictionaries (including OrderedDict) return "view" objects from their keys()
# and values() methods. Those are iterable, but don't support indexing.In Python 2, keys() and values() returned lists.

# So we have to convert them to lists to access their index as follows:
# items = list(df.items())
# items[2]

# But we need our worksheets to be a dataframe not list element so we follow the following approach of creating indexes
# ourselves.
# We are creating indexes for worksheets to make their use easier in the further analysis like in loops it is
# necessary to have indexes associated with the elements. We can't loop over distinct names.

In [44]:
# for i in range(0,len(df)):
#     df[i] = pd.read_excel('C:/Users/user/Downloads/INFORM COVID-19 RISK INDEX v012.xlsx',sheet_name=i, skiprows = range(1))
df[0] = df['INFORM COVID-19 RISK 2020 (a-z)']
df[1] = df['Hazard & Exposure'] 
df[2] = df['Vulnerability']
df[3] = df['Lack of Coping Capacity']

In [45]:
df.keys()

odict_keys(['INFORM COVID-19 RISK 2020 (a-z)', 'Hazard & Exposure', 'Vulnerability', 'Lack of Coping Capacity', 0, 1, 2, 3])

In [46]:
# df[0].head(5)

In [47]:
# df[0].tail(5)

In [48]:
# Rows (0,192:195) are not required for the analysis. They just carry explanatory information.

In [49]:
# df[1].head(5)

In [50]:
# df[1].tail(5)

In [51]:
# Last two rows (191, 192) are different from rest of the data and contain just the reference values for calculations for 
# calculating the indexes in the table.

In [52]:
# df[2].head(5)

In [53]:
# df[2].tail(5)

In [54]:
# Last two rows (191, 192) are different from rest of the data and contain just the reference values for calculations for 
# calculating the indexes in the table.

In [55]:
# df[3].head(5)

In [56]:
# df[3].tail(5)

In [57]:
# Last two rows (191, 192) are different from rest of the data and contain just the reference values for calculations for 
# calculating the indexes in the table.

In [58]:
# The rows 0 and 192:195 in the sheet 'INFORM COVID-19 RISK 2020 (a-z)' df[0] are redundant for the analysis. Hence, dropping them.

In [59]:
# df[0].reset_index(inplace=True)
df[0].drop(np.r_[0,192:196,], axis=0, inplace=True) # Upper bound (196) not included
# Since we dropped index 0 we have to reset indexes else the row index in df[0] would start from 1 conrary to other data frames 
# where indexes would start from 0. Removing rows from end doesn't create any difference to indexes so we can skip resetting the
# indexes if the rows are just dropped from the end.
df[0].reset_index(drop=True, inplace=True)

In [60]:
# Since same index of last two rows (191 and 192) are not required for analysis in all the three sheets, So we delete them in all
# the three sheets together using for loop.

In [61]:
for i in range(1,4):
    df[i].drop([191,192], inplace=True, axis=0)

In [62]:
# Also "Country" and "ISO3" columns are common in all the sheets and should be used as key to join all these 3 tables.
# Just that "ISO3" is "Iso3" in sheet 'Hazard & Exposure'/ 0, so we need to rename it, to have the same names in all the three 
# sheets for the common columns on which we want to join them.

In [63]:
df[1].rename(columns={"Iso3":"ISO3"}, inplace=True)

In [64]:
# Only keeping the required columns in the dataframes

In [65]:
# df[0] ('INFORM COVID-19 RISK 2020 (a-z)') is kind of a summary dataframe for all other 3 dataframes, so it has all the main 
# variables which df[1] ('Hazard & Exposure'), df[2] ('Vulnerability'), df[3] ('Lack of Coping Capacity') have
# but few of the aggregate columns are created/derived in this dataframe itself from the variables of other three data frames,
# so just keeping the new derived columns in this dataframe df[0] because other columns we would pick from their corresponding 
# dataframes

df[0] = df[0][['COUNTRY','VULNERABILITY (Hazard-independent)','VULNERABILITY',
'LACK OF COPING CAPACITY (Hazard-independent)', 'LACK OF COPING CAPACITY', "INFORM COVID-19 RISK", "COVID-19 RISK CLASS"]]

df[1] = df[1][['COUNTRY','WaSH','Population', 'P2P']]

df[2] = df[2][['COUNTRY','Development & Deprivation','Inequality','Economic Dependency Index','INFORM Socio-Economic Vulnerability',
'Uprooted people','Health Conditions','Food Security','GBV', 'INFORM Vulnerable Groups',
'Movements','Behaviour','Demographic and Co-morbidities', 'Covid-19 Vulnerability']]

df[3] = df[3][['COUNTRY','Corruption Perception Index','Government Effectiveness','INFORM Institutional',
'Immunization coverage','per capita public and private expenditure on health care','Maternal Mortality ratio', 'INFORM Infrastructure',
'IHR', 'Operational readiness index','Covid-19 Lack of coping capacity']]


In [66]:
# Since the dataframes we made 0, 1, 2, 3 are the copies of workbook sheets 'INFORM COVID-19 RISK 2020 (a-z)','Hazard & Exposure',
# 'Vulnerability','Lack of Coping Capacity'

# So since we made changes on 0, 1, 2 and 3 , we will replicate those changes to 'INFORM COVID-19 RISK 2020 (a-z)',
# 'Hazard & Exposure', 'Vulnerability','Lack of Coping Capacity' sheets as follows, so that they are identical.
# 0 with 'INFORM COVID-19 RISK 2020 (a-z)'; 1 with 'Hazard & Exposure'; 2 with 'Vulnerability' and 3 with 
# 'Lack of Coping Capacity'.

In [67]:
df['INFORM COVID-19 RISK 2020 (a-z)']= df[0]
df['Hazard & Exposure']= df[1] 
df['Vulnerability']= df[2] 
df['Lack of Coping Capacity']= df[3]

In [68]:
# df[0]

In [69]:
# Now joining the 3 sheets in to 1, we can do it either using actual names or indexes, because they are identical.
# 0 with 'Hazard & Exposure'; 1 with 'Vulnerability', 2 with 'Lack of Coping Capacity'.

In [70]:
df_combine = df
for i in range(1,4):
    df_combine[0] = pd.merge(df_combine[0], df_combine[i], how="inner", on=["COUNTRY"])
df_risk = df_combine[0]

In [71]:
df_risk.head(3)

,COUNTRY,VULNERABILITY (Hazard-independent),VULNERABILITY,LACK OF COPING CAPACITY (Hazard-independent),LACK OF COPING CAPACITY,INFORM COVID-19 RISK,COVID-19 RISK CLASS,WaSH,Population,P2P,...,Corruption Perception Index,Government Effectiveness,INFORM Institutional,Immunization coverage,per capita public and private expenditure on health care,Maternal Mortality ratio,INFORM Infrastructure,IHR,Operational readiness index,Covid-19 Lack of coping capacity
0,Afghanistan,8.2,6.4,8.3,7.7,6.8,Very High,6.0,6.6,6.4,...,8.4,7.9,8.2,7.2,9.6,7.1,8.3,6.5,7.5,7
1,Albania,1.9,4.1,4.8,4.9,4.2,Medium,0.9,5.0,3.6,...,6.5,4.8,5.7,0.0666667,7.6,0.2,3.7,x,5,5
2,Algeria,3,3.6,5.2,3.9,3.9,Medium,1.4,5.7,4.3,...,6.5,5.9,6.2,1.43333,6.8,1.2,4,2,2.5,2.25


In [72]:
# df_risk.columns.values
# or
# for cols in df_risk.columns:
#     print(cols)

In [73]:
df_risk = df_risk.rename(columns={"COUNTRY": "Country/Region"})
df_risk = df_risk.set_index("Country/Region")
df_risk.head(3)

,VULNERABILITY (Hazard-independent),VULNERABILITY,LACK OF COPING CAPACITY (Hazard-independent),LACK OF COPING CAPACITY,INFORM COVID-19 RISK,COVID-19 RISK CLASS,WaSH,Population,P2P,Development & Deprivation,...,Corruption Perception Index,Government Effectiveness,INFORM Institutional,Immunization coverage,per capita public and private expenditure on health care,Maternal Mortality ratio,INFORM Infrastructure,IHR,Operational readiness index,Covid-19 Lack of coping capacity
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,8.2,6.4,8.3,7.7,6.8,Very High,6.0,6.6,6.4,8.6,...,8.4,7.9,8.2,7.2,9.6,7.1,8.3,6.5,7.5,7
Albania,1.9,4.1,4.8,4.9,4.2,Medium,0.9,5.0,3.6,1.9,...,6.5,4.8,5.7,0.0666667,7.6,0.2,3.7,x,5,5
Algeria,3,3.6,5.2,3.9,3.9,Medium,1.4,5.7,4.3,3.1,...,6.5,5.9,6.2,1.43333,6.8,1.2,4,2,2.5,2.25


In [74]:
df_risk.shape

(191, 32)

In [75]:
# Renaming the columns to common name prefixes for the ones under same category


df_risk = df_risk.rename(columns={'P2P':'P2P',
                'WaSH': 'P2P_1','Population': 'P2P_2', 
                
                
                'VULNERABILITY': 'Vul',
                'VULNERABILITY (Hazard-independent)':'Vul_HI',
                
                'INFORM Socio-Economic Vulnerability':'Vul_1',
                'Development & Deprivation': 'Vul_11', 'Inequality':'Vul_12' ,
                'Economic Dependency Index':'Vul_13',
                
                'INFORM Vulnerable Groups': 'Vul_2',
                'Uprooted people':'Vul_21','Health Conditions':'Vul_22','Food Security':'Vul_23','GBV':'Vul_24',             
                
                'Covid-19 Vulnerability': 'Vul_3/Vul_HD',
                'Movements':'Vul_31','Behaviour': 'Vul_32','Demographic and Co-morbidities': 'Vul_33',                 
                
                
                'LACK OF COPING CAPACITY' : 'LOCC',
                'LACK OF COPING CAPACITY (Hazard-independent)': 'LOCC_HI',
                
                
                'INFORM Institutional': 'LOCC_1',
                'Corruption Perception Index': 'LOCC_11','Government Effectiveness': 'LOCC_12',
                
                'INFORM Infrastructure': 'LOCC_2',
                'Immunization coverage':'LOCC_21',
                'per capita public and private expenditure on health care':'LOCC_22',
                'Maternal Mortality ratio': 'LOCC_23', 
                
                
                'Covid-19 Lack of coping capacity': 'LOCC_3/LOCC_HD',
                'IHR': 'LOCC_31', 'Operational readiness index': 'LOCC_32',
                                
               })

# Rearranging the columns: To put related ones together in a easy to inerpret sequence

df_risk = df_risk[['P2P','P2P_1','P2P_2', 
              
              'Vul','Vul_HI',
              'Vul_1','Vul_11', 'Vul_12','Vul_13',
              'Vul_2','Vul_21','Vul_22','Vul_23','Vul_24',             
              'Vul_3/Vul_HD','Vul_31','Vul_32','Vul_33',                 
              
              'LOCC','LOCC_HI',
              'LOCC_1','LOCC_11','LOCC_12',
              'LOCC_2','LOCC_21','LOCC_22','LOCC_23', 
              'LOCC_3/LOCC_HD','LOCC_31', 'LOCC_32',
                  
              'INFORM COVID-19 RISK',
              'COVID-19 RISK CLASS']]

# df_risk.shape
df_risk.head(3)

,P2P,P2P_1,P2P_2,Vul,Vul_HI,Vul_1,Vul_11,Vul_12,Vul_13,Vul_2,...,LOCC_12,LOCC_2,LOCC_21,LOCC_22,LOCC_23,LOCC_3/LOCC_HD,LOCC_31,LOCC_32,INFORM COVID-19 RISK,COVID-19 RISK CLASS
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,6.4,6.0,6.6,6.4,8.2,7.9,8.6,7.7,6.8,8.5,...,7.9,8.3,7.2,9.6,7.1,7,6.5,7.5,6.8,Very High
Albania,3.6,0.9,5.0,4.1,1.9,2.1,1.9,2.1,2.3,1.7,...,4.8,3.7,0.0666667,7.6,0.2,5,x,5,4.2,Medium
Algeria,4.3,1.4,5.7,3.6,3,2.5,3.1,3.3,0.3,3.4,...,5.9,4,1.43333,6.8,1.2,2.25,2,2.5,3.9,Medium


#### Preparing the Risk data to be merged with the "Confirmed Cases"/"Deaths"/"Recovered" Cases datasets by aligning the "Country/Region" column with other datasets 

In [76]:
# gr_confirmed1.head(3)
# gr_confirmed.head(3)

In [77]:
check_f = pd.merge(gr_confirmed, df_risk, how="outer", on="Country/Region", indicator=True)
check_f[check_f['_merge'] == 'right_only']

,1,2,3,4,5,6,7,8,9,10,...,LOCC_2,LOCC_21,LOCC_22,LOCC_23,LOCC_3/LOCC_HD,LOCC_31,LOCC_32,INFORM COVID-19 RISK,COVID-19 RISK CLASS,_merge
Country/Region,,,,,,,,,,,,,,,,,,,,,
Brunei Darussalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.6,0.1,4,0.3,2.5,x,2.5,3.1,Low,right_only
Congo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9,5.35,9.3,4.2,5.15,5.3,5,5.7,High,right_only
Congo DR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,3.25,10,5.3,5.8,6.6,5,6.8,Very High,right_only
Côte d'Ivoire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.8,1.25,9.6,6.9,5.3,5.6,5,5.6,High,right_only
Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,1,1.7,0,2.9,3.3,2.5,3.4,Low,right_only
Kiribati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.3,2.1,9.3,1,6.2,4.9,7.5,5.9,High,right_only
Korea DPR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6,0.25,x,1,4.35,3.7,5,4.7,Medium,right_only
Korea Republic of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,0.233333,1,0.1,0.3,0.6,0,2.9,Low,right_only
Lao PDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.7,2.55,9.6,2.1,7,6.5,7.5,4.6,Medium,right_only


In [78]:
# After comparing the index column (Country/Region) in gr_confirmed (Confirmed cases dataset), which is representative of distinct names in 
# gr_deaths (deaths dataset), gr_recovered (recovered cases dataset) and df_test (testing data), we find that 
# there are some difference in names of the same countries between df_death and df_risk
# Total 25 rows which seem to be only in Risk data but out of them 10 are just named differently in the risk data in comparison
# to their names in Confirmed/Deaths/Recovered datasets. So naming them in alignment with these datasets as follows:

In [79]:
df_risk = df_risk.rename(index={'Brunei Darussalam': 'Brunei', "Congo": "Congo (Brazzaville)", 
                                          "Congo DR": "Congo (Kinshasa)", "Czech Republic": "Czechia",
                                         "Korea Republic of":"Korea, South", "Lao PDR":"Laos",
                                         "Moldova Republic of": "Moldova","Russian Federation":"Russia",
                                         "United States of America": "US", "Viet Nam": "Vietnam"})
check_f = pd.merge(gr_confirmed, df_risk, how="outer", on="Country/Region", indicator=True)
check_f[check_f['_merge'] == 'right_only']
# df_risk.head(3)

,1,2,3,4,5,6,7,8,9,10,...,LOCC_2,LOCC_21,LOCC_22,LOCC_23,LOCC_3/LOCC_HD,LOCC_31,LOCC_32,INFORM COVID-19 RISK,COVID-19 RISK CLASS,_merge
Country/Region,,,,,,,,,,,,,,,,,,,,,
Côte d'Ivoire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.8,1.25,9.6,6.9,5.3,5.6,5,5.6,High,right_only
Kiribati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.3,2.1,9.3,1,6.2,4.9,7.5,5.9,High,right_only
Korea DPR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6,0.25,x,1,4.35,3.7,5,4.7,Medium,right_only
Marshall Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9,4.7,7,x,6.3,5.1,7.5,5.4,High,right_only
Micronesia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.3,6.06667,8.7,1,7.5,x,7.5,5,High,right_only
Myanmar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.7,2.2,9.2,2.8,4.2,3.4,5,4.8,Medium,right_only
Nauru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.2,1.4,5.9,x,7.05,6.6,7.5,5.2,High,right_only
Palau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.9,0.9,3.8,x,5.75,4,7.5,3.9,Medium,right_only
Palestine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,0,x,0.3,7.5,x,7.5,5,High,right_only


Above 15 are altogether not there in Confirmed/Deaths/Recovered dataset (by any othr name also) but are only in risk index data.

### Merging the 4 datasets: 
#### confirmed, deaths, recovered, and testing

In [80]:
# gr_confirmed_melt.shape
# gr_death_melt.shape
# gr_recovered_melt.shape
# gr_recovered_melt.shape
# df_test_melt.shape

In [81]:
df1 = gr_confirmed_melt.reset_index()
df2 = gr_death_melt.reset_index()
df3 = gr_recovered_melt.reset_index()
df4 = df_test_melt.reset_index()
# df5 = df_risk

In [82]:
data = [df1, df2, df3, df4]
len(data)
for i in range(1,(len(data))): # loop is for len(data)-1 time, because upper bound is not included
    data[0] = pd.merge(data[0], data[i], how="inner", on=["Country/Region", "Days"])
MergedData = data[0]
MergedData.set_index("Country/Region", inplace = True)

In [83]:
MergedData.to_csv("C:/Users/user/Downloads/COVID-19 Related info/Created Files/Merged_Data.csv")
MergedData.head(50) # 26180 Rows, 5 Columns

,Days,Confirmed Cases,Deaths,Recovered Cases,Testing Count
Country/Region,,,,,
Afghanistan,1,5.0,0.0,0.0,0.0
Afghanistan,2,7.0,0.0,0.0,0.0
Afghanistan,3,7.0,0.0,0.0,0.0
Afghanistan,4,7.0,0.0,0.0,0.0
Afghanistan,5,11.0,0.0,0.0,0.0
Afghanistan,6,16.0,0.0,0.0,0.0
Afghanistan,7,21.0,0.0,1.0,0.0
Afghanistan,8,22.0,0.0,1.0,0.0
Afghanistan,9,22.0,0.0,1.0,0.0


In [84]:
df_risk.to_csv("C:/Users/user/Downloads/COVID-19 Related info/Created Files/Risk_Data_Filtered.csv")
df_risk.head(20) # 191 rows, 30 columns

,P2P,P2P_1,P2P_2,Vul,Vul_HI,Vul_1,Vul_11,Vul_12,Vul_13,Vul_2,...,LOCC_12,LOCC_2,LOCC_21,LOCC_22,LOCC_23,LOCC_3/LOCC_HD,LOCC_31,LOCC_32,INFORM COVID-19 RISK,COVID-19 RISK CLASS
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,6.4,6.0,6.6,6.4,8.2,7.9,8.6,7.7,6.8,8.5,...,7.9,8.3,7.2,9.6,7.1,7,6.5,7.5,6.8,Very High
Albania,3.6,0.9,5.0,4.1,1.9,2.1,1.9,2.1,2.3,1.7,...,4.8,3.7,0.0666667,7.6,0.2,5,x,5,4.2,Medium
Algeria,4.3,1.4,5.7,3.6,3,2.5,3.1,3.3,0.3,3.4,...,5.9,4,1.43333,6.8,1.2,2.25,2,2.5,3.9,Medium
Angola,6.5,6.8,6.4,4.3,5.4,5.6,8.1,6.1,0.1,5.2,...,7.1,7.4,8.26667,9.5,2.7,4.55,4.1,5,5.6,High
Antigua and Barbuda,2.8,1.0,3.7,3.4,2.7,2.9,2.5,x,3.7,2.4,...,5.0,3.7,3,6.9,0.5,5.4,5.8,5,3.6,Medium
Argentina,2.8,0.4,4.0,3.2,1.6,1.8,1.4,4.4,0.0,1.4,...,4.9,2.5,2.6,5,0.4,4.3,3.6,5,3.4,Low
Armenia,2.7,0.4,3.9,4.4,2.1,2.0,1.5,2.8,2.2,2.1,...,5.0,3,0.633333,7.2,0.3,3.35,1.7,5,3.6,Medium
Australia,2.5,0.0,3.8,3.5,1.4,0.5,0.0,2.1,0.0,2.2,...,1.8,1,0.833333,0,0.1,0.5,1,0,2.1,Low
Austria,2.5,0.0,3.8,4.7,1.5,0.3,0.0,1.2,0.1,2.6,...,2.1,0.6,2,0,0.1,3.3,4.1,2.5,3,Low
